In [1]:
import io
import re
import glob
import datetime
from functools import partial

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

blanklabel = "B:16"
normlabel = "B:14"
date = "20240916"
datafile = "M_20240916-204829 b.next Envision FLUOR deGFP Timecourse.csv"
label_order = [f"B:{2*i+2}" for i in range(8)]

In [2]:
def read_platemap_str(platemap_str: str) -> pd.DataFrame:
    platemap = pd.read_table(io.StringIO(platemap_str), index_col=0)
    platemap.index.name = "Row"
    platemap = platemap.reset_index().melt(id_vars=["Row"], var_name="Column", value_name="Construct")
    
    platemap["Column"] = platemap["Column"].astype(int)
    platemap["Well"] = platemap.apply(lambda well: f"{well['Row']}:{well['Column']}", axis=1)
    platemap = platemap.rename(columns={"Construct": "Label"})
    platemap = platemap.dropna()
    return platemap

def read_platemap_excel(platemap_path: str) -> pd.DataFrame:
    """
    Use like this:

    
    > platemap_path = "path/to/platemap.xlsx"
    > platemap = read_platemap_excel(platemap_path)
    > platemap.head()
    """
    platemap = pd.read_excel(platemap_path)
    platemap.fillna(value=0, inplace=True)
    platemap['Row'] = platemap['Well'].apply(lambda s: s.split(":")[0])
    platemap['Column'] = platemap['Well'].apply(lambda s: s.split(":")[1]).astype(int)

    return platemap

# LOAD YOUR PLATEMAP HERE
platemap = read_platemap_excel(
    f"{date}-platemap.xlsx"
)
platemap.head()

,Well,Sample(Ribosome/P-mix),Ribosome,Energy-Mix,Protein-Mix,DNA,Row,Column
0,B:2,bnext/bnext,bnext-01-01,NEB-Sol-A,bnext-03-02,deGFP,B,2
1,B:4,NEB/bnext,NEB,NEB-Sol-A,bnext-03-02,deGFP,B,4
2,B:6,bnext/NEB,bnext-01-01,NEB-Sol-A,NEB-Sol-B-delta-ribosome,deGFP,B,6
3,B:8,bnext(+)/NEB,bnext-01-01,NEB-Sol-A,NEB-Sol-B-delta-ribosome,deGFP,B,8
4,B:10,NEB/NEB,NEB,NEB-Sol-A,NEB-Sol-B-delta-ribosome,deGFP,B,10


In [4]:
def read_envision(
    datafile: str, 
    platemap: pd.DataFrame,
    blanklabel: str = False
) -> pd.DataFrame:
    # load data
    data = pd.read_csv(datafile)
    # massage Row, Column, and Well information
    data["Row"] = data["Well ID"].apply(lambda s: s[0])
    data["Column"] = data["Well ID"].apply(lambda s: str(int(s[1:])))
    data["Well"] = data.apply(lambda well: f"{well['Row']}:{well['Column']}", axis=1)
    # merge on Well
    data = data.merge(
        platemap.drop(["Row", "Column"], axis=1), 
        left_on="Well", right_on="Well", how="inner"
    )
    
    data["Data"] = data["Result Channel 1"]
    data["Ex"] = data["Exc WL[nm]"]
    data["Em"] = data["Ems WL Channel 1[nm]"]
    data["Wavelength"] = data["Ex"] + "," + data["Em"]

    data['Time'] = pd.to_timedelta(data['Time [hhh:mm:ss.sss]']).astype('timedelta64[s]')
    data['Seconds'] = data['Time'].map(lambda x: x.total_seconds())

    # apply blanking, if blanklabel given
    if blanklabel:
        num_conditions = len(set(data["Well"]))
        
        # make a new DataFrame with just the blanks; use "Repeat" as time variable
        background_column = data[data["Well"] == blanklabel]
        background_data = background_column["Data"]
        background_repeat = background_column["Repeat"]
        background = pd.DataFrame({
            "Background": background_data,
            "Repeat": background_repeat
        })
        
        # merge blanks dataframe with data; subtract
        data = pd.merge(data, background, on="Repeat")
        data["BackgroundSubtracted"] = data["Data"] - data["Background"]
        
    return data

data = read_envision(
    datafile="M_20240916-204829 b.next Envision FLUOR deGFP Timecourse.csv",
    platemap=platemap,
    blanklabel=blanklabel
)
data.head()

,Operation,Plate Barcode,Loop,Repeat,Well ID,Row,Column,Sample Type,Time [hhh:mm:ss.sss],Measurement Time,...,Protein-Mix,DNA,Data,Ex,Em,Wavelength,Time,Seconds,Background,BackgroundSubtracted
0,Fluorescence Intensity Filter 1,V-2024-09-16 20:48:26,1,1,B02,B,2,Undefined,000:00:00.712,2024-09-16 20:48:38 -07:00,...,bnext-03-02,deGFP,1346305,485 / 20,535 / 40,"485 / 20,535 / 40",0 days 00:00:00,0.0,1359366,-13061
1,Fluorescence Intensity Filter 1,V-2024-09-16 20:48:26,1,1,B04,B,4,Undefined,000:00:02.122,2024-09-16 20:48:39 -07:00,...,bnext-03-02,deGFP,1373934,485 / 20,535 / 40,"485 / 20,535 / 40",0 days 00:00:02,2.0,1359366,14568
2,Fluorescence Intensity Filter 1,V-2024-09-16 20:48:26,1,1,B06,B,6,Undefined,000:00:03.540,2024-09-16 20:48:40 -07:00,...,NEB-Sol-B-delta-ribosome,deGFP,1353360,485 / 20,535 / 40,"485 / 20,535 / 40",0 days 00:00:03,3.0,1359366,-6006
3,Fluorescence Intensity Filter 1,V-2024-09-16 20:48:26,1,1,B08,B,8,Undefined,000:00:04.950,2024-09-16 20:48:42 -07:00,...,NEB-Sol-B-delta-ribosome,deGFP,1348033,485 / 20,535 / 40,"485 / 20,535 / 40",0 days 00:00:04,4.0,1359366,-11333
4,Fluorescence Intensity Filter 1,V-2024-09-16 20:48:26,1,1,B10,B,10,Undefined,000:00:06.361,2024-09-16 20:48:43 -07:00,...,NEB-Sol-B-delta-ribosome,deGFP,1399010,485 / 20,535 / 40,"485 / 20,535 / 40",0 days 00:00:06,6.0,1359366,39644


In [5]:
sns.lineplot(
    data=data,
    x='Time',
    y='BackgroundSubtracted',
    hue="Sample(Ribosome/P-mix)"
)
plt.savefig(f"figures/{date}-traces-overview.png", dpi=300)

NameError: name 'sns' is not defined